# NB EMA

1) organizza il notebook in funzioni diverse, di modo che puoi ripetere le analisi su più di un dataset
2) esporta in csv il nuovo dataset, magari con le analisi fatte e pusha 
3) comincia a pensare alla normalizzazione 
4) vedi se questi cazzo di dati sono block sparse oppure no 


###  DS lab 2019

## Behavioural data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl

In [ ]:
# import data into csv file

alldata_beh = pd.read_csv('data/Behavioral/AllData.csv')

### Nan percentages

In [52]:
def find_nan_percentages(dataframe): #determining NaN percentages in the dataframe, returns dict ith nan percentages
    
    nan_percentages = []    # [COLUMN_NAME, PERCENTAGE OF NANs]
    for column in dataframe: 
        n_elements = 0
        n_nan_elements = 0
        for row in dataframe.index: 
            elem = dataframe.loc[row, column]
            if pd.isna(elem)==True :
                n_nan_elements+=1
            n_elements+=1
        nan_percentages.append([column, round((n_nan_elements/n_elements),2)*100])
    nan_perc = dict(nan_percentages) #NaN percentages dict
    
    return nan_perc



In [ ]:
#isolate behavioral test data : no id info, nor diagnoses

beh_test_data = alldata_beh[ list(alldata_beh.loc[:,'ACE_01':]) ] 
beh_test_data

In [55]:
# find features within a threshold of NaN percentage, returns a list of the features

def find_frequent_features(dataframe, perc_threshold):
    nan_perc = find_nan_percentages(dataframe)
    freq_features = []   #features with less than 50% NaN 
    for feature in dataframe.columns: 
        if nan_perc.get(feature) < perc_threshold: 
            freq_features.append(feature)
    return freq_features
    

#btdata_f = beh_test_data.loc[:,freq_features]
        

In [ ]:
# finding basic features : id, basics, diagnoses

basic_columns = alldata_beh[ list(alldata_beh.loc[:,:'Days_Baseline'])].columns  #columns with id, basic features and diagnoses
basic_features = []       #list with basic features 
for col in basic_columns: 
    basic_features.append(col)


In [ ]:
# filter alldata_beh with only basic and frequent features

allbdata_f = alldata_beh.loc[:, (basic_features+freq_features)]  #filtered alldata
allbdata_f  



In [ ]:
# exporting in csv 

# file = pd.to_csv('path')

### Nan percentages range in dataset

In [47]:
def nan_perc_ranges(dataframe): #returns list of Nan perc ranges, with associated features
    
    nan_percentages = find_nan_percentages(dataframe)
    features_perc_range = []   #list with percentages of NaN ranges associated with features
                               # [10, list_of_features] element means that labels associated are from 10% to 19.99% of NaN 
    for threshold in np.arange(10):
        feature_list = []
        for feature, perc in nan_percentages.items():
            if perc >= threshold*10.0 and perc <= (threshold+1)*10.0: 
                feature_list.append(feature)
        features_perc_range.append([threshold*10,feature_list])
        
    return features_perc_range

In [ ]:
# printing NaN percentages ranges with associated features
features_perc_range = nan_perc_ranges(alldata_beh)
for perc,features in features_perc_range:
    print('number of features in', perc,'% - ', perc+10,'% range :', len(features))

## Matching columns with descriptions

In [ ]:
# reading valid column names from the description files 

import glob

path = 'data/Behavioral/description' 
desc_files = glob.glob(path + "/*.xlsx")

varname_cols = []
desc_errors = 0
wrong_desc = []
processed_files = [] #list of processed files to avoid errors

for filename in desc_files:
    df = pd.read_excel(filename, skiprows=[0]) #index_col=None, header=0
    column_names = df.columns
    #print(column_names)
    if 'Variable Name' in column_names: 
        varname_cols.append(df.loc[:,'Variable Name'])
    elif 'Variable' in column_names: 
        varname_cols.append(df.loc[:,'Variable'])
    else:
        desc_errors += 1
        wrong_desc.append(filename)
    processed_files.append(filename)

print([file for file in processed_files])
        
'''
print ('description that do not contain a column that matches are :', desc_errors ,'\n',
       'the wrong descriptions are ',
        [d for d in wrong_desc])
        
FOUND THAT 'data/Behavioral/description/ESWAN.xlsx' RAISES PROBLEMS. NAIVE CORRECTION
'''

# fix issue with ESWAN 

path_ESWAN = 'data/Behavioral/description/ESWAN.xlsx'
df = pd.read_excel(path_ESWAN, skiprows=[0,1]) #index_col=None, header=0
#column_names = df.columns
varname_cols.append(df.loc[:,'Variable Name'])

# fix issue with nested directory 

path_inside_nested_dir = 'data/Behavioral/description/*'
desc_files_nested = glob.glob(path_inside_nested_dir + "/*.xlsx")
for filepath in desc_files_nested:
    df = pd.read_excel(filepath, skiprows=[0])
    varname_cols.append(df.loc[:,'Variable Name'])

# resulting list

print ('the resulting list has lenght', len (varname_cols))  #outputs 123. works. 




In [ ]:
varnames_df = pd.DataFrame(np.hstack([i.values for i in varname_cols])) 
varnames_df = varnames_df.dropna()    #drop nan
varnames = varnames_df.values  # numpy array with variable names from descriptions
varnames_list = [elem[0] for elem in varnames.tolist()]  #list with variable names from descriptions

In [ ]:
beh_data_matched = alldata_beh.copy(deep=True)   # dataframe with behavioural data keeping only basic columns and the ones mentioned in the decriptions
for column in beh_data_matched.columns: 
    if column not in varnames_list and column not in basic_features:
        beh_data_matched.drop(columns=column, inplace=True)

In [ ]:
beh_data_matched

In [ ]:
#converting to strings as the original data is, before exporting in csv
beh_data_matched = beh_data_matched.astype(str)

In [ ]:
# exporting in csv behavioral data matched
file = beh_data_matched.to_csv('data/Behavioral/behdata_matched_ema.csv')

## Working with cleaned (behavioural) data

In [3]:
# importing clean data 

finalSummaries = pd.read_csv('data/Behavioral/cleaned/finalSummaries.csv')
finalSummariesLabels = pd.read_csv('data/Behavioral/cleaned/finalSummariesLabels.csv')
HBNFinalSummaries = pd.read_csv('data/Behavioral/cleaned/HBNFinalSummaries.csv')

/anaconda3/envs/dslab_1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (53,83,85,97,98,99,100,101,112,113,114,115,116,127,128,129,131,142,143,144,146,263,278,279,282,283,284,287,288,289,292,293,294) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cleandata = [finalSummaries, finalSummariesLabels, HBNFinalSummaries]

# shapes
for dataset in cleandata: 
    print(dataset.shape)
    

(326, 1)
(334, 2)
(2096, 399)


In [43]:
# why do finalSummaries, finalSummariesLabels differ in row size? 

non_matched_labels = []
for label in finalSummaries.loc[:,'x']: 
    if label not in finalSummariesLabels.loc[:,'label'].values:
        non_matched_labels.append(label)
print('number of non-matched labels: ',len(non_matched_labels),'\n')

dupli = pd.DataFrame(finalSummariesLabels.duplicated(subset=['label']))

duplicate_labels = []
for index in dupli.index[dupli[0]==True].to_list(): 
    duplicate_labels.append(finalSummariesLabels.loc[index, 'label'])

print(finalSummariesLabels[finalSummariesLabels['label'].isin(duplicate_labels)] )
    

number of non-matched labels:  0 

           label                                   question
1020   CELF_WC_R                     Word Classes Raw Score
1023   CELF_FD_R             Following Directions Raw Score
1026   CELF_FS_R             Formulated Sentences Raw Score
1029   CELF_RS_R              Recalling Sentences Raw Score
1032  CELF_USP_R  Understanding Spoken Paragraphs Raw Score
1053   CELF_WC_R                     Word Classes Raw Score
1056   CELF_FD_R             Following Directions Raw Score
1059   CELF_FS_R             Formulated Sentences Raw Score
1062   CELF_RS_R              Recalling Sentences Raw Score
1065  CELF_USP_R  Understanding Spoken Paragraphs Raw Score
5689     SWAN_IN                        Inattention Average
5690     SWAN_HY                      Hyperactivity Average
5691  SWAN_Total                               SWAN Average
5711     SWAN_IN                        Inattention Average
5712     SWAN_HY                      Hyperactivity Average
5713 

There are no labels in finalSummaries that are not in finalSummariesLabels. 

In [45]:
HBNFinalSummaries.loc[:,duplicate_labels]

/anaconda3/envs/dslab_1/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,CELF_WC_R,CELF_FD_R,CELF_FS_R,CELF_RS_R,CELF_USP_R,SWAN_IN,SWAN_HY,SWAN_Total
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# nan percentages in HBNSummaries

HBN_nanpercranges = nan_perc_ranges(HBNFinalSummaries)

In [54]:
# printing NaN percentages ranges with associated features
for perc,features in HBN_nanpercranges:
    print('number of features in', perc,'% - ', perc+10,'% range :', len(features))

number of features in 0 % -  10 % range : 13
number of features in 10 % -  20 % range : 41
number of features in 20 % -  30 % range : 47
number of features in 30 % -  40 % range : 34
number of features in 40 % -  50 % range : 15
number of features in 50 % -  60 % range : 8
number of features in 60 % -  70 % range : 16
number of features in 70 % -  80 % range : 18
number of features in 80 % -  90 % range : 36
number of features in 90 % -  100 % range : 189


In [57]:
HBN_reduced = HBNFinalSummaries.loc[:,find_frequent_features(HBNFinalSummaries, 90.0)] # HBNFinalSummaries reduced to columns with less than 90% Nan

In [58]:
HBN_reduced

,Anonymized.ID,EID,Sex,Age,Study.Site,NoDX,DX_01_Cat,DX_01_Sub,DX_01,DX_01_Spec,...,WISC_PSI,WISC_FSIQ,YSR_AB,YSR_AD,YSR_WD,YSR_RBB,YSR_SC,YSR_Ext,YSR_Int,YSR_Total
1,A00078864,NDARYM832PX3,1,7.048254,1,Yes,Anxiety Disorders,NaN,Generalized Anxiety Disorder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A00078865,NDARNJ687DMC,1,6.348163,1,Yes,Depressive Disorders,NaN,Disruptive Mood Dysregulation Disorder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A00078866,NDARRM363BXZ,0,10.052589,1,Yes,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A00078867,NDARUW586LLL,1,12.319415,1,Yes,Depressive Disorders,NaN,Major Depressive Disorder,Single episode,...,NaN,NaN,9.0,12.0,8.0,7.0,9.0,16.0,29.0,85.0
5,A00078868,NDARDC298NW4,0,13.901437,1,Yes,Neurodevelopmental Disorders,Intellectual Disability,Intellectual Disability-Mild,NaN,...,NaN,NaN,8.0,10.0,5.0,2.0,11.0,10.0,26.0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,A00093557,NDARYZ986HEW,1,7.563084,3,Yes,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,NaN,...,116.0,129.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2093,A00093558,NDARPM572ZZV,0,9.832762,3,Yes,Neurodevelopmental Disorders,Specific Learning Disorder,Specific Learning Disorder with Impairment in ...,rule out,...,98.0,112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,A00093559,NDARYA955CY1,0,15.357860,3,Yes,Neurodevelopmental Disorders,Specific Learning Disorder,Specific Learning Disorder with Impairment in ...,NaN,...,92.0,111.0,5.0,5.0,4.0,6.0,4.0,11.0,13.0,47.0
2095,A00093560,NDARRT283RHB,1,10.712070,1,Dropped out before diagnosis was given,No Diagnosis Given: Incomplete Eval,NaN,No Diagnosis Given: Incomplete Eval,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
HBNFinalSummaries.shape

(2096, 399)

In [60]:
HBN_reduced

,Anonymized.ID,EID,Sex,Age,Study.Site,NoDX,DX_01_Cat,DX_01_Sub,DX_01,DX_01_Spec,...,WISC_PSI,WISC_FSIQ,YSR_AB,YSR_AD,YSR_WD,YSR_RBB,YSR_SC,YSR_Ext,YSR_Int,YSR_Total
1,A00078864,NDARYM832PX3,1,7.048254,1,Yes,Anxiety Disorders,NaN,Generalized Anxiety Disorder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A00078865,NDARNJ687DMC,1,6.348163,1,Yes,Depressive Disorders,NaN,Disruptive Mood Dysregulation Disorder,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A00078866,NDARRM363BXZ,0,10.052589,1,Yes,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A00078867,NDARUW586LLL,1,12.319415,1,Yes,Depressive Disorders,NaN,Major Depressive Disorder,Single episode,...,NaN,NaN,9.0,12.0,8.0,7.0,9.0,16.0,29.0,85.0
5,A00078868,NDARDC298NW4,0,13.901437,1,Yes,Neurodevelopmental Disorders,Intellectual Disability,Intellectual Disability-Mild,NaN,...,NaN,NaN,8.0,10.0,5.0,2.0,11.0,10.0,26.0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,A00093557,NDARYZ986HEW,1,7.563084,3,Yes,Neurodevelopmental Disorders,Attention-Deficit/Hyperactivity Disorder,ADHD-Combined Type,NaN,...,116.0,129.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2093,A00093558,NDARPM572ZZV,0,9.832762,3,Yes,Neurodevelopmental Disorders,Specific Learning Disorder,Specific Learning Disorder with Impairment in ...,rule out,...,98.0,112.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,A00093559,NDARYA955CY1,0,15.357860,3,Yes,Neurodevelopmental Disorders,Specific Learning Disorder,Specific Learning Disorder with Impairment in ...,NaN,...,92.0,111.0,5.0,5.0,4.0,6.0,4.0,11.0,13.0,47.0
2095,A00093560,NDARRT283RHB,1,10.712070,1,Dropped out before diagnosis was given,No Diagnosis Given: Incomplete Eval,NaN,No Diagnosis Given: Incomplete Eval,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
